In [1]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
import numpy as np

In [2]:
data_ctx = mx.cpu()
model_ctx = mx.cpu()
# model_ctx = mx.gpu()

In [12]:
batch_size = 64
num_inputs = 784
num_outputs = 10
num_examples = 60000
def transform(data, label):
    return data.astype(np.float32)/255, label.astype(np.float32)
train_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=True, transform=transform),
                                      batch_size, shuffle=True)
test_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=False, transform=transform),
                              batch_size, shuffle=False)

In [15]:
net = gluon.nn.Dense(num_outputs)
net.collect_params().initialize(mx.init.Normal(sigma=1.), ctx=model_ctx)

In [16]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

In [17]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(model_ctx).reshape((-1,784))
        label = label.as_in_context(model_ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [18]:
evaluate_accuracy(test_data, net)

0.058999999999999997

In [19]:
epochs = 10
moving_loss = 0.

for e in range(epochs):
    cumulative_loss = 0
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(model_ctx).reshape((-1,784))
        label = label.as_in_context(model_ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)
        cumulative_loss += nd.sum(loss).asscalar()

    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, cumulative_loss/num_examples, train_accuracy, test_accuracy))

Epoch 0. Loss: 2.22572903929, Train_acc 0.7924, Test_acc 0.8004
Epoch 1. Loss: 0.874186870948, Train_acc 0.835983333333, Test_acc 0.8412
Epoch 2. Loss: 0.714929977957, Train_acc 0.854716666667, Test_acc 0.856
Epoch 3. Loss: 0.633576641377, Train_acc 0.8631, Test_acc 0.8645
Epoch 4. Loss: 0.581123975619, Train_acc 0.872, Test_acc 0.8708
Epoch 5. Loss: 0.543082058756, Train_acc 0.877233333333, Test_acc 0.877
Epoch 6. Loss: 0.513986707719, Train_acc 0.88265, Test_acc 0.8818
Epoch 7. Loss: 0.490974807866, Train_acc 0.885683333333, Test_acc 0.8842
Epoch 8. Loss: 0.471334000023, Train_acc 0.88835, Test_acc 0.8848
Epoch 9. Loss: 0.455235342979, Train_acc 0.890916666667, Test_acc 0.8889
